### 1. VNC (Virtual Network Computing)

VNC создает виртуальный рабочий стол, к которому вы можете подключиться с вашего локального компьютера.

**Шаги на VPS (через SSH):**

1.  **Установите графическую среду (если ее нет):**
    Если у вас минимальная установка Ubuntu, вам нужно установить графическую среду, например GNOME, XFCE или LXDE. XFCE и LXDE легче и потребляют меньше ресурсов, что хорошо для VPS.
    * **XFCE (рекомендуется для VPS):**
        ```bash
        sudo apt update
        sudo apt install xfce4
        ```
    * **GNOME (более ресурсоемкий):**
        ```bash
        sudo apt update
        sudo apt install ubuntu-desktop
        ```

2.  **Установите VNC-сервер:**
    Наиболее популярный - `tightvncserver` или `tigervnc-standalone-server`.
    ```bash
    sudo apt install tightvncserver
    # или
    sudo apt install tigervnc-standalone-server
    ```

3.  **Запустите VNC-сервер и настройте пароль:**
    ```bash
    vncserver
    ```
    При первом запуске вам будет предложено установить пароль для VNC-подключения. Запомните его. Также он покажет номер дисплея (например, `:1`).

4.  **Остановите VNC-сервер для конфигурации:**
    ```bash
    vncserver -kill :1  # Замените :1 на ваш номер дисплея
    ```

5.  **Настройте VNC-сервер для использования XFCE (или вашей графической среды):**
    Отредактируйте файл `~/.vnc/xstartup`:
    ```bash
    nano ~/.vnc/xstartup
    ```
    Удалите все содержимое файла и вставьте следующее (для XFCE):
    ```bash
    #!/bin/bash
    xrdb $HOME/.Xresources
    startxfce4 &
    ```
    Сделайте файл исполняемым:
    ```bash
    chmod +x ~/.vnc/xstartup
    ```

6.  **Запустите VNC-сервер снова:**
    ```bash
    vncserver
    ```

7.  **Откройте порт VNC в брандмауэре (если используется UFW):**
    Порт для VNC обычно `5900 + номер дисплея`. Например, для дисплея `:1` это будет `5901`.
    ```bash
    sudo ufw allow 5901/tcp
    ```

**Шаги на локальном компьютере:**

1.  **Установите VNC-клиент:**
    Популярные клиенты: RealVNC Viewer, TightVNC Viewer, Remmina (для Linux).

2.  **Подключитесь к VPS:**
    Откройте ваш VNC-клиент и введите IP-адрес вашего VPS с номером дисплея (например, `your_vps_ip:1` или `your_vps_ip::5901`). Введите пароль, который вы установили на VPS.

### 2. X11 Forwarding (перенаправление X-сервера)

Этот метод позволяет запускать отдельные графические приложения с вашего VPS и отображать их на вашем локальном компьютере. Он не предоставляет полный рабочий стол, но удобен для запуска отдельных программ.

**Шаги на VPS (через SSH):**

1.  **Убедитесь, что `xauth` установлен:**
    ```bash
    sudo apt install xauth
    ```

2.  **Разрешите X11Forwarding в конфигурации SSH:**
    Отредактируйте файл `/etc/ssh/sshd_config`:
    ```bash
    sudo nano /etc/ssh/sshd_config
    ```
    Найдите строку `X11Forwarding` и убедитесь, что она раскомментирована и установлена в `yes`:
    ```
    X11Forwarding yes
    ```
    Найдите строку `X11DisplayOffset` и убедитесь, что она раскомментирована (если есть).
    Найдите строку `X11UseLocalhost` и установите ее в `no` (если есть).
    ```
    X11DisplayOffset 10
    X11UseLocalhost no
    ```
    Сохраните файл и перезапустите SSH-сервис:
    ```bash
    sudo systemctl restart ssh
    ```

**Шаги на локальном компьютере:**

1.  **Для Linux/macOS:**
    Вы можете использовать SSH-клиент напрямую. Убедитесь, что у вас установлен X-сервер (например, XQuartz для macOS).
    Подключитесь к VPS с опцией `-X` или `-Y` (для доверенного перенаправления):
    ```bash
    ssh -X user@your_vps_ip
    # или
    ssh -Y user@your_vps_ip
    ```
    После подключения вы можете запустить графическое приложение, например:
    ```bash
    firefox &
    # или
    gedit &
    ```
    Оно должно открыться на вашем локальном компьютере.

2.  **Для Windows:**
    Вам потребуется PuTTY для SSH-подключения и Xming (или VcXsrv) в качестве X-сервера.
    * **Установите Xming.**
    * **Запустите Xming.**
    * **Настройте PuTTY:**
        * В PuTTY, в категории "Connection" -> "SSH" -> "X11", установите флажок "Enable X11 forwarding".
        * В поле "X display location" оставьте пустым или установите "localhost:0".
        * Подключитесь к вашему VPS как обычно.
    * После подключения, вы можете запустить графическое приложение, и оно должно появиться на вашем рабочем столе Windows.

### Какой метод выбрать?

* **VNC:** Идеально подходит, если вам нужен полноценный удаленный рабочий стол, как будто вы сидите за компьютером. Он также продолжает работать, даже если вы отключите SSH-сессию.
* **X11 Forwarding:** Отлично подходит, если вам нужно запускать только несколько графических приложений, не загружая VPS полноценным рабочим столом. Менее ресурсоемкий.

Для большинства пользователей, которым нужен "рабочий стол", **VNC** будет более подходящим вариантом.